# IBM Data Science Professional Certificate
## Applied Data Science Capstone Project

### Description
#### Opening a new gym in Toronto
The purpose of the analysis is to identify neighborhoods in Toronto that to open a new private gym. This analysis will begin with web scraping neighborhoods in Toronto and the popular venues in different neighborhoods, and distinguish neighborhoods with great potential to open a new private gym using machine learning techniques.

#### 1. Import Libraries

In [2]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import requests 
from bs4 import BeautifulSoup 

import json 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print("Libraries imported.")

Libraries imported.


#### 2. Web Scraping the Neighborhoods in Toronto

In [3]:
# Create a beautiful soup object

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

In [4]:
# Loading data to the 3 lists

postalcode_lst = []
borough_lst = []
neighborhood_lst = []

# Append data to the lists created above
for tr in soup.find_all('tr')[1:]:
    tds = tr.find_all('td')
    try: 
        postalcode_lst.append(tds[0].text[:-1])
        borough_lst.append(tds[1].text[:-1])
        neighborhood_lst.append(tds[2].text[:-1])
    except Exception as e:
        pass

    
postalcode_lst = postalcode_lst[:180]
borough_lst = borough_lst[:180]
neighborhood_lst = neighborhood_lst[:180]

In [6]:
# checking the length of the 3 lists and the neighbourhood list
print(len(postalcode_lst))
print(len(borough_lst))
print(len(neighborhood_lst))

180
180
180


In [21]:
# Creating a neighborhood dataframe
columns = ['PostalCode','Borough','Neighborhood']
neigh_df = pd.DataFrame(columns=columns)

neigh_df['PostalCode'] = postalcode_lst
neigh_df['Borough'] = borough_lst
neigh_df['Neighborhood'] = neighborhood_lst

neigh_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [22]:
# Investigating the Not assigned values in Borough
neigh_df.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [23]:
# Investigating the Not assigned values in Neighborhood
neigh_df.Neighborhood.value_counts()

Not assigned                                                                                                                              77
Downsview                                                                                                                                  4
Don Mills                                                                                                                                  2
Scarborough Village                                                                                                                        1
Forest Hill North & West, Forest Hill Road Park                                                                                            1
Bathurst Manor, Wilson Heights, Downsview North                                                                                            1
Steeles West, L'Amoreaux West                                                                                                              1
Dufferin, Dov

In [24]:
# Removing all Not assgined records
old_n = neigh_df.shape[0]
neigh_df = neigh_df[neigh_df.Borough != 'Not assigned']
neigh_df = neigh_df[neigh_df.Neighborhood != 'Not assigned']
new_n = neigh_df.shape[0]
print (str(old_n - new_n) + " recrods are removed")

77 recrods are removed


#### 3. Obtaining Latitude and Longitude information

In [25]:
geo_df = pd.read_csv("http://cocl.us/Geospatial_data")
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [26]:
df = pd.merge(neigh_df, geo_df, left_on='PostalCode', right_on='Postal Code')
df = df.drop('Postal Code', axis  = 1)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### 4. Visualizing the Boroughs in Toronto 

In [30]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 5. Using Four Square API to Explore Popular Venues in Different Neighborhoods in Toronto 

In [217]:
import getpass
CLIENT_ID = getpass.getpass('Enter your Client ID') 
CLIENT_SECRET = getpass.getpass('Enter your Client Secret') 
VERSION = '20180605' 
LIMIT = 100
radius = 1000

Enter your Client ID········
Enter your Client Secret········


In [218]:
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [246]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [247]:
print(toronto_venues.shape)
toronto_venues.head()

(4987, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [221]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop                                 386
Café                                        203
Park                                        158
Restaurant                                  145
Pizza Place                                 144
Italian Restaurant                          117
Bakery                                      111
Grocery Store                               101
Japanese Restaurant                          83
Sandwich Place                               83
Gym                                          77
Bank                                         75
Sushi Restaurant                             75
Hotel                                        71
Pharmacy                                     65
Fast Food Restaurant                         64
Bar                                          61
Pub                                          61
Thai Restaurant                              59
Gastropub                                    54
Indian Restaurant                       

In [248]:
toronto_venues_lst = sorted(toronto_venues['Venue Category'].unique().tolist())

In [249]:
for i in toronto_venues_lst:
    if 'gym' in i.lower():
        print (i)

Boxing Gym
Climbing Gym
College Gym
Gym
Gym / Fitness Center
Gym Pool


From the list of venues, it is apparent that there are two categories of gym to be considered in the analysis, whch are Gym and Gym / Fitness Center. Therefore these two have to be combined and classifed as the same category.

In [250]:
toronto_venues['Venue Category'].value_counts()[['Gym', 'Gym / Fitness Center']]

Gym                     77
Gym / Fitness Center    32
Name: Venue Category, dtype: int64

In [251]:
# Replacing Gym / Fitness Center to Gym
toronto_venues['Venue Category'].replace('Gym / Fitness Center', 'Gym', inplace = True)

In [252]:
toronto_venues['Venue Category'].value_counts()[['Gym']]

Gym    109
Name: Venue Category, dtype: int64

In [253]:
toronto_venues.shape

(4987, 7)

In [254]:
toronto_venues['Venue Category'].value_counts()[['Neighborhood']]

Neighborhood    14
Name: Venue Category, dtype: int64

In [255]:
toronto_venues['Venue Category'].replace('Neighborhood', 'Community', inplace = True)

In [256]:
toronto_venues['Venue Category'].value_counts()[['Community']]

Community    14
Name: Venue Category, dtype: int64

#### 6. Encoding Venues

In [258]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(4987, 336)


,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smok

In [259]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(99, 336)

In [233]:
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smok

In [260]:
len(toronto_venues['Neighborhood'].unique())

99

In [261]:
len(toronto_grouped[toronto_grouped['Gym'] > 0])

54

In [262]:
toronto_grouped['Gym'].describe().to_frame()

,Gym
count,99.000000
mean,0.020659
std,0.026258
min,0.000000
25%,0.000000
50%,0.010000
75%,0.030000
max,0.125000


In [263]:
n = round (len(toronto_grouped[toronto_grouped['Gym'] > 0]) / len(toronto_venues['Neighborhood'].unique())*100 , 2)
print (str(n) + '% of neighborhoods has at least a gym in the neighborhood')

54.55% of neighborhoods has at least a gym in the neighborhood


#### 7. Clutering Neighborhoods into 5 clusters based on prevalence of gym

In [264]:
# Create a dataframe containing Neighbourhoods and Gym
toronto_gym = toronto_grouped[['Neighborhood','Gym']]

In [265]:
toronto_gym

,Neighborhood,Gym
0,Agincourt,0.000000
1,"Alderwood, Long Branch",0.038462
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000
3,Bayview Village,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000
5,Berczy Park,0.030000
6,"Birch Cliff, Cliffside West",0.058824
7,"Brockton, Parkdale Village, Exhibition Place",0.020000
8,"Business reply mail Processing Centre, South C...",0.020000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000


In [291]:
# set number of clusters
kclusters = 5

toronto_gym_clustering = toronto_gym.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_gym_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 3, 3, 3, 0, 2, 4, 4, 3], dtype=int32)

In [292]:
toronto_gym['Cluster Labels'] = kmeans.labels_

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [293]:
toronto_gym

,Neighborhood,Gym,Cluster Labels
0,Agincourt,0.000000,3
1,"Alderwood, Long Branch",0.038462,4
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,3
3,Bayview Village,0.000000,3
4,"Bedford Park, Lawrence Manor East",0.000000,3
5,Berczy Park,0.030000,0
6,"Birch Cliff, Cliffside West",0.058824,2
7,"Brockton, Parkdale Village, Exhibition Place",0.020000,4
8,"Business reply mail Processing Centre, South C...",0.020000,4
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,3


In [294]:
neigh_df = df[['Neighborhood', 'Latitude', 'Longitude']]

In [295]:
neigh_df

,Neighborhood,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656
1,Victoria Village,43.725882,-79.315572
2,"Regent Park, Harbourfront",43.654260,-79.360636
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,"Malvern, Rouge",43.806686,-79.194353
7,Don Mills,43.745906,-79.352188
8,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,"Garden District, Ryerson",43.657162,-79.378937


In [296]:
toronto_merged = toronto_gym.join(neigh_df.set_index('Neighborhood'), on='Neighborhood')

In [297]:
toronto_merged

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Agincourt,0.000000,3,43.794200,-79.262029
1,"Alderwood, Long Branch",0.038462,4,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,3,43.754328,-79.442259
3,Bayview Village,0.000000,3,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0.000000,3,43.733283,-79.419750
5,Berczy Park,0.030000,0,43.644771,-79.373306
6,"Birch Cliff, Cliffside West",0.058824,2,43.692657,-79.264848
7,"Brockton, Parkdale Village, Exhibition Place",0.020000,4,43.636847,-79.428191
8,"Business reply mail Processing Centre, South C...",0.020000,4,43.662744,-79.321558
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,3,43.628947,-79.394420


In [298]:
cluster = toronto_merged.groupby('Cluster Labels').Gym.agg(['mean']
                                                          ).sort_values('mean').rename(columns={'mean':'Gym Mean'})

In [299]:
cluster

,Gym Mean
Cluster Labels,
3,0.006849
1,0.022275
4,0.029426
0,0.035258
2,0.067376


From the above summary, it is apparent that Cluster 2 has the lowest prevalence of gym, which indicates a good investment opporunity

In [300]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 8. Explore and Visualize Neighborhoods in Cluster 2 

In [303]:
toronto_target = toronto_merged[toronto_merged['Cluster Labels'] ==3]
toronto_target = toronto_target.reset_index(drop = True)
toronto_target

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Agincourt,0.000000,3,43.794200,-79.262029
1,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,3,43.754328,-79.442259
2,Bayview Village,0.000000,3,43.786947,-79.385975
3,"Bedford Park, Lawrence Manor East",0.000000,3,43.733283,-79.419750
4,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,3,43.628947,-79.394420
5,Cedarbrae,0.000000,3,43.773136,-79.239476
6,Central Bay Street,0.000000,3,43.657952,-79.387383
7,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,3,43.781638,-79.304302
8,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,3,43.716316,-79.239476
9,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,3,43.691116,-79.476013


In [304]:
map_toronto_target = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, label in zip(toronto_target['Latitude'], toronto_target['Longitude'], toronto_target['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_target)  
    
map_toronto_target